# 标定

In [56]:
import numpy as np
import cv2 as cv
import glob
import os
from Stitcher import Stitcher
import numpy as np

In [26]:
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points
objp = np.zeros((8*11,3), np.float32)
objp[:,:2] = np.mgrid[0:11,0:8].T.reshape(-1,2)

## 摄像头标定函数

In [27]:
toBeCalibratedImages_1 = glob.glob(r"D:/imageStitching/1206/camera1/*.jpg")
saveLocation_1 = r"D:/imageStitching/1206/camera1_calibrated/"
dstLocation_1 = r"D:/imageStitching/1206/camera1_undistorted/"

toBeCalibratedImages_2 = glob.glob(r"D:/imageStitching/1206/camera2/*.jpg")
saveLocation_2 = r"D:/imageStitching/1206/camera2_calibrated/"
dstLocation_2 = r"D:/imageStitching/1206/camera2_undistorted/"

In [28]:
def calibration(toBeCalibratedImages,saveLocation,dstLocation):
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points
objp = np.zeros((8*11,3), np.float32)
objp[:,:2] = np.mgrid[0:11,0:8].T.reshape(-1,2)
    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
    if not os.path.exists(dstLocation):
        os.makedirs(dstLocation)
    
    
    i=0
    for fname in toBeCalibratedImages:
        i=i+1
        img = cv.imread(fname)
        # print(fname.index)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        # Find the chess board corners
        ret, corners = cv.findChessboardCorners(gray, (11,8), None)
        # If found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners)
            # Draw and display the corners
            cv.drawChessboardCorners(img, (11,8), corners2, ret)
            cv.imshow('img', img)
            cv.imwrite(saveLocation + str(i) + '.jpg' , img)
            cv.waitKey(500)
        ret1, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        h, w = img.shape[:2]
        newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

        # undistort 1
        dst = cv.undistort(img, mtx, dist, None, newcameramtx)
        # crop the image
        x, y, w, h = roi
        dst = dst[y:y + h, x:x + w]
        cv.imshow('dst', dst)
        cv.imwrite(dstLocation + '1_' + str(i) + '.jpg', dst)
        cv.waitKey(500)

        # # undistort 2
        # mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w, h), 5)
        # dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
        # # crop the image
        # x, y, w, h = roi
        # dst = dst[y:y + h, x:x + w]
        # cv.imshow('dst', dst)
        # cv.imwrite(dstLocation + '2_' + str(i) + '.jpg', dst)
        # cv.waitKey(500)

    cv.destroyAllWindows()
    return (ret1, mtx, dist, rvecs, tvecs)

## 帧去畸变函数

In [29]:
def undistort (img , mtx , dist):
    h, w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
    # undistort
    dst = cv.undistort(img, mtx, dist, None, newcameramtx)
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y + h, x:x + w]
    return dst

## videoflow

In [57]:
# 读取视频
s1 = cv.VideoCapture(1)
s2 = cv.VideoCapture(2)
# s3 = cv2.VideoCapture(3)

In [58]:
# 读视频第一帧
success1, frame1 = s1.read()
success2, frame2 = s2.read()

In [59]:
success1

True

In [62]:
success2

False

In [60]:
print(frame1)


[[[10  3  4]
  [12  3  4]
  [14  2  4]
  ...
  [10  3  3]
  [10  3  4]
  [10  3  4]]

 [[10  3  4]
  [10  3  4]
  [10  3  4]
  ...
  [10  3  4]
  [10  3  4]
  [10  3  4]]

 [[14  2  4]
  [12  3  4]
  [10  3  4]
  ...
  [10  3  4]
  [10  3  4]
  [10  3  4]]

 ...

 [[ 8  3  4]
  [ 8  3  4]
  [ 8  3  4]
  ...
  [ 9  3  4]
  [10  3  4]
  [10  3  4]]

 [[ 8  3  4]
  [ 8  3  4]
  [ 8  3  4]
  ...
  [ 5  1  1]
  [ 8  3  4]
  [ 8  3  4]]

 [[ 8  3  4]
  [ 9  3  4]
  [10  3  4]
  ...
  [ 5  1  1]
  [ 8  3  4]
  [ 8  3  4]]]


In [61]:
print(frame1.shape)
print(frame1.size)
print(frame1.dtype)

print(frame2.shape)
print(frame2.size)
print(frame2.dtype)

(480, 640, 3)
921600
uint8


AttributeError: 'NoneType' object has no attribute 'shape'

In [38]:
frame1 = frame1[0:480,80:560]
frame2 = frame2[0:480,80:560]
stitcher = Stitcher()
(result, vis) = stitcher.stitchleft([frame1, frame2], showMatches=True)
resultInfo = result.shape
size = (resultInfo[1],resultInfo[0])

# 获取帧率
fps1 = s1.get(cv.CAP_PROP_FPS)
fps2 = s2.get(cv.CAP_PROP_FPS)

# VideoWriter实例化
fourcc = cv.VideoWriter_fourcc('X','V','I','D')
writer = cv.VideoWriter('1.avi', fourcc , 5 , size )
result.dtype = np.uint8
writer.write(result)
i=0;j=0;

#摄像头标定
(ret1_1, mtx_1, dist_1, rvecs_1, tvecs_1)=calibration(toBeCalibratedImages_1,saveLocation_1,dstLocation_1)
(ret1_2, mtx_2, dist_2, rvecs_2, tvecs_2)=calibration(toBeCalibratedImages_2,saveLocation_2,dstLocation_2)

while success1:
    i=i+1
    success1, frame1 = s1.read()
    if frame1 is None:
        print("frame1 is none, break")
        break
    frame1=frame1[0:480,80:560]
    frame1=undistort(frame1,mtx_1, dist_1)
    print ("processing: ",i,",success1:",success1,"frame1",frame1[320,180,:])

    while success2:
        j=j+1
        success2, frame2 = s2.read()
        if frame2 is None:
            print("frame2 is none, break")
            break
        frame2=frame2[0:480,80:560]
        frame2=undistort(frame2,mtx_2, dist_2)
        print("processing: ", j, ",success2:", success2, "frame2", frame2[320, 180, :])

        (result, vis) = stitcher.stitchleft([frame1, frame2], showMatches=True)

        print("result:",result[320,180,:])
        result.dtype = np.uint8
        writer.write(result)

        cv.imshow("result", result);
        # if (cv.waitKey(3) & 0xff) == ord('q'):
        if (cv.waitKey(1) & 0xff) == 27:
            break
        break
    if (cv.waitKey(1) & 0xff) == 27:
        break

# 释放VideoCapture资源
s2.release()
s1.release()
# 释放VideoWriter资源
writer.release()
print("successfully released!")
cv.destroyAllWindows()

TypeError: 'NoneType' object is not subscriptable